In [1]:
import pandas as pd

In [2]:
data=pd.read_csv("train.csv")

In [3]:
data.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
test=pd.read_csv("test.csv")

In [5]:
test_final=test["PassengerId"]
def clean(data):
    data=data.drop(["Ticket","Name","PassengerId"],axis=1)

    cols=["SibSp","Parch","Fare","Age"]
    for col in cols:
        data.fillna({col:data[col].mean()},inplace=True)

    data.fillna({"Embarked":"U"},inplace=True)
    return data

data=clean(data)
test=clean(test)

In [6]:
from sklearn import preprocessing 
le=preprocessing.LabelEncoder()

cols=["Sex","Embarked"]

for col in cols:
    data[col]=le.fit_transform(data[col])
    test[col]=le.transform(test[col])

In [7]:
def cabin_letter(col:pd.Series)->pd.Series:
    
    letters=col.astype('string').str.upper().str.extract(r'([A-Z])',expand=False)

    codes=pd.Series(0,index=col.index,dtype='int64')
    mask=letters.notna()
    
    if mask.any():
        codes[mask]=le.fit_transform(letters[mask])+1
    return codes


data["Cabin"]=cabin_letter(data["Cabin"])
test["Cabin"]=cabin_letter(test["Cabin"])

In [8]:
y=data["Survived"]
X=data.drop("Survived",axis=1)

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.55,random_state=350)

In [9]:
import xgboost as xgb
xg_reg=xgb.XGBClassifier(objective='binary:logistic')
xg_reg.fit(X_train,y_train)
prediction=xg_reg.predict(X_test)


In [10]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,prediction)

0.8207739307535642

In [11]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform,randint

param_dist= {
    'max_depth':randint(3,10),
    'learning_rate':uniform(0.01,0.2),
    'n_estimators':randint(100,500),
    'subsample':uniform(0.7,0.3),
    'colsample_bytree':uniform(0.7,0.3),
    'gamma':uniform(0,1),
    'reg_alpha':uniform(0,1),
    'reg_lambda':uniform(1,2)}
Xgb2=xgb.XGBClassifier(eval_metric='logloss')

random_search=RandomizedSearchCV(
    estimator=Xgb2,
    param_distributions=param_dist,
    n_iter=50,
    scoring='accuracy',
    cv=5,
    verbose=1,
    n_jobs=-1)

random_search.fit(X_train,y_train)
print("Best parameters:",random_search.best_params_)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best parameters: {'colsample_bytree': np.float64(0.9532948739595792), 'gamma': np.float64(0.6516065909285761), 'learning_rate': np.float64(0.024442946787302425), 'max_depth': 9, 'n_estimators': 137, 'reg_alpha': np.float64(0.7968482197158628), 'reg_lambda': np.float64(1.2402968334432984), 'subsample': np.float64(0.8072108626293678)}


In [12]:
import numpy as np
xgbfinal=xgb.XGBClassifier(
    max_depth=3,
    learning_rate=np.float64(0.02141249059198768),
    n_estimators=210,
    subsample=np.float64(0.9027411895511758),
    colsample_bytree=np.float64(0.7623835854455334),
    gamma=np.float64(0.69077460518946),
    reg_alpha=np.float64(0.37418870122476044),
    reg_lambda=np.float64(2.289227559574978),eval_metric='logloss')

xgbfinal.fit(X_train,y_train)
prediction_final=xgbfinal.predict(X_test)
accuracy_score(y_test,prediction_final)

0.8228105906313645

Exception ignored in: <function ResourceTracker.__del__ at 0x10f0aff60>
Traceback (most recent call last):
  File "/Users/maudq/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/Users/maudq/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/Users/maudq/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes
Exception ignored in: <function ResourceTracker.__del__ at 0x108423f60>
Traceback (most recent call last):
  File "/Users/maudq/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/Users/maudq/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/Users/maudq/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes
Exception ignored in: <function 

In [ ]:
submission_prediction=xgbfinal.predict(test)

In [ ]:
df=pd.DataFrame({"PassengerId":test_final.values,
                 "Survived":submission_prediction})
df.to_csv("submission.csv",index=False)